# Introduction

The requirements for undergraduate CS certificate can be found at https://guide.wisc.edu/undergraduate/letters-science/computer-sciences/computer-sciences-ba/#requirementstext. 

# Mathematical Model

### Notations

$T$ is the max number of maximum number of semesters (e.g., $T = 6$ means that we only consider 3 years)

$C$ is the set of classes (e.g., $C = \{ \text{CS 200}, \text{CS 300}, \text{CS 524}, \cdots \}$)


### Decision variables

$x[t, c]$ is a Boolean variable to denote whether to take the class $c \in C$ at semester $t \in \{1, \cdots, T\}$. 

If a student takes CS 200 on his/her first semester, then $x[1, \text{CS 200}] = 1$

If a student takes CS 524 on the second Spring semester, then $x[3, \text{CS 524}]= 1$


### Prerequisite constraint

To take CS 300, the student need to take CS 200 in the previous semesters, we can encode such prerequisite using the following constraint: 

$$x[t, \text{CS 300}] \le \sum_{i=1}^{t - 1} x[i, \text{CS 200}] $$


CS 524 has the prerequisite similar to "(CS 200 or 300) and (MATH 340 or 341)", we can encode it with the constraint:

$$x[t, \text{CS 524}] \le \sum_{i=1}^{t - 1} x[i, \text{CS 200}] + \sum_{i=1}^{t - 1} x[i, \text{CS 300}] $$

$$x[t, \text{CS 524}] \le \sum_{i=1}^{t - 1} x[i, \text{CS 340}] + \sum_{i=1}^{t - 1} x[i, \text{CS 341}] $$


### Graduation requirement

In order to meet the graduation requirement, CS 300 must be taken, so we have the constraint:

$$\sum_{t=1}^{T} x[t, \text{CS 300}] \ge 1$$
    
There are other kinds of graduation requirements, such as taking $k$ coureses from a list of $C$, we can encode such requirements using the following constraint:

$$\sum_{t=1}^{T} \sum_{c \in C} x[t, c] \ge k$$
    
### Max credits

For undergraduate, we can take at most 18 credits in a given semester, so we need to ensure that

$$\sum_{c \in C} x[t, c] \cdot \text{credit}(c) \le 18, \quad \text{for all semester } t$$

### Do not retake class

Generally, we don't want to take the same class multiple times, so we have the cons

### Objective 

A naive objective would be to minimize the sum of $x$: 

$$\min \sum_{i=t}^{T} \sum_{c \in C} x[t, c]$$

But this would lead to many equally good solutions. To avoid this, we add a weight $t$ to the class $c$ if its taken at semester $t$

$$\min \sum_{i=t}^{T} \sum_{c \in C} x[t, c] \cdot t$$

This avoid students procrastinating class to later semesters. 

# Data Preparation

We first load the json file obtained from the UW-Madison [Course Search & Enrollment website](enroll.wisc.edu), and extract all the CS courses.

In [7]:
using JSON

raw_data = JSON.parsefile("/work/CS524-Spring-2021-Final-Project/data/1214-spring-2021.json")["hits"]

cls_data = Dict(
    Symbol(c["courseDesignation"]) => 
    c for c in raw_data 
    if c["subject"]["shortDescription"] == "COMP SCI" && parse(Int, c["catalogSort"]) <= 700
)

cls = keys(cls_data)
n_cls = length(cls_data)

println("We have ", string(n_cls), " undergraduate courses in total: ", join(sort([String(e) for e in cls]), ", "))

We have 57 undergraduate courses in total: COMP SCI 200, COMP SCI 220, COMP SCI 240, COMP SCI 252, COMP SCI 270, COMP SCI 298, COMP SCI 300, COMP SCI 304, COMP SCI 310, COMP SCI 319, COMP SCI 320, COMP SCI 352, COMP SCI 354, COMP SCI 368, COMP SCI 369, COMP SCI 400, COMP SCI 402, COMP SCI 407, COMP SCI 412, COMP SCI 425, COMP SCI 435, COMP SCI 475, COMP SCI 502, COMP SCI 506, COMP SCI 513, COMP SCI 514, COMP SCI 520, COMP SCI 524, COMP SCI 525, COMP SCI 532, COMP SCI 533, COMP SCI 534, COMP SCI 536, COMP SCI 537, COMP SCI 538, COMP SCI 539, COMP SCI 540, COMP SCI 542, COMP SCI 552, COMP SCI 558, COMP SCI 559, COMP SCI 564, COMP SCI 567, COMP SCI 570, COMP SCI 576, COMP SCI 577, COMP SCI 579, COMP SCI 611, COMP SCI 612, COMP SCI 638, COMP SCI 639, COMP SCI 640, COMP SCI 642, COMP SCI 679, COMP SCI 681, COMP SCI 682, COMP SCI 699


In [6]:
show(IOContext(stdout, :limit => false), "text/plain", cls_data[Symbol("COMP SCI 524")]);
# 
# join(keys(cls_data[Symbol("COMP SCI 524")]), " ")

Dict{String,Any} with 44 entries:
  "honors" => nothing
  "allCrossListedSubjects" => Any[Dict{String,Any}("departmentURI"=>"http://www.cs.wisc.edu/","footnotes"=>Any["Courses taught and managed by the Computer Sciences department often have enrollment restrictions that give students in UW-Madison Computer Sciences programs priority access during initial enrollment periods.\n\nEvening exams are likely for most of our undergraduate courses."],"formalDescription"=>"COMPUTER SCIENCES","undergraduateCatalogURI"=>"http://guide.wisc.edu/undergraduate/letters-science/computer-sciences/","termCode"=>"1214","departmentOwnerAcademicOrgCode"=>"L0780","description"=>"COMPUTER SCIENCES","graduateCatalogURI"=>"http://guide.wisc.edu/graduate/computer-sciences/","uddsFundingSource"=>"A4820","shortDescription"=>"COMP SCI","subjectCode"=>"266","schoolCollege"=>Dict{String,Any}("schoolCollegeURI"=>"http://www.ls.wisc.edu/","shortDescription"=>"Letters and Science","formalDescription"=>"Letters and Scienc

In [173]:
for (k, v) in sort(cls_data)
    println(k, ": ", v["enrollmentPrerequisites"])
end

COMP SCI 200: Satisfied Quantitative Reasoning (QR) A requirement
COMP SCI 220: Satisfied Quantitative Reasoning (QR) A requirement or declared in the Professional Capstone Program in Computer Sciences. Not open to students with credit for COMP SCI 301.
COMP SCI 240: MATH 217, 221, or 275
COMP SCI 252: None
COMP SCI 270: Not open to students with credit for COMP SCI 570
COMP SCI 298: Consent of instructor
COMP SCI 300: Satisfied QR-A requirement and (COMP SCI 200, 220, 302, 310, or 301) or (COMP SCI/E C E 252 and E C E 203); graduate/professional standing; or declared in the Capstone Certificate in Computer Sciences. Not open to students with credit for COMP SCI 367.
COMP SCI 304: Concurrent enrollment in COMP SCI 200 and member of Wisconsin Emerging Scholars program in Computer Science
COMP SCI 310: MATH 222, graduate/professional standing, or declared in the Capstone Certificate in Computer Sciences for Professionals
COMP SCI 319: Graduate/professional standing
COMP SCI 320: COMP SCI

# Constructing the Optimization Problem

In [201]:
using JuMP, Cbc

max_credit = 18
n_sem = 4
T = 1:n_sem
C = cls

function credit(c)
    return cls_data[c]["minimumCredits"]
end

function make_cs_symbol(c)
    return Symbol("COMP SCI ", c)
end

function cs_lst(arr...)
    res = [make_cs_symbol(c) for c in arr]
    return [c for c in res if c in C]
end

function add_cs_prereq(c, prereq...)
    for t in T
        @constraint(m, 
            x[make_cs_symbol(c), t] <= sum(x[make_cs_symbol(p), i] for i in 1:t-1 for p in prereq)
        )
    end
end

C_basic = cs_lst(240, 252, 300, 354, 400)
C_theory = cs_lst(577, 520)
C_xware = cs_lst(407, 506, 536, 538, 537, 552, 564, 640, 642)
C_app = cs_lst(412, 425, 513, 514, 524, 525, 534, 540, 545, 547, 559, 570)
C_elec = cs_lst(407, 412, 425, 435, 471, 475, 506, 513, 514, 520, 524, 525, 526, 532, 533, 534, 536, 537, 538, 539, 540, 545, 547, 552, 558, 559, 564, 567, 570, 576, 577, 579, 635, 640, 642, 679, 639)

# m = Model(Cbc.Optimizer)
m = Model(with_optimizer(Cbc.Optimizer, logLevel=0))

@variable(m, x[C, T], Bin)

# Take all from basic computer sciences
for c in C_basic
    @constraint(m, sum(x[c, t] for t in T) >= 1)
end

# Complete 1 for Theory of computer science
@constraint(m, sum(x[c, t] for t in T for c in C_theory) >= 1)

# Complete 2 for Software & Hardware
@constraint(m, sum(x[c, t] for t in T for c in C_xware) >= 2)

# Complete 1 for Applications
@constraint(m, sum(x[c, t] for t in T for c in C_app) >= 1)

# Complete 2 for Electives
@constraint(m, sum(x[c, t] for t in T for c in C_elec) >= 2)

# Max credit constraint
for t in T
    @constraint(m, sum(x[c, t] * credit(c) for c in C) <= max_credit)
end

# Prerequisites
add_cs_prereq(300, 200)

add_cs_prereq(354, 252)
add_cs_prereq(354, 300)

add_cs_prereq(400, 300)
add_cs_prereq(407, 300)

add_cs_prereq(506, 400)
add_cs_prereq(506, 407, 536, 537, 559, 564, 570, 679, 552)

add_cs_prereq(514, 200, 300)
add_cs_prereq(513, 200, 300)

add_cs_prereq(534, 300)

add_cs_prereq(536, 354)
add_cs_prereq(536, 400)

add_cs_prereq(537, 354)
add_cs_prereq(537, 400)

add_cs_prereq(538, 354)
add_cs_prereq(538, 400)

add_cs_prereq(540, 300)

add_cs_prereq(520, 400)
add_cs_prereq(520, 240, 475)

add_cs_prereq(552, 352)
add_cs_prereq(552, 354)

add_cs_prereq(564, 354)
add_cs_prereq(564, 400)

add_cs_prereq(570, 200, 300)

add_cs_prereq(577, 400)
add_cs_prereq(577, 240, 475)

add_cs_prereq(640, 537)

add_cs_prereq(642, 537)



    
@objective(m, Min, sum(x[c, t] * t for c in C, t in T))


optimize!(m)
println(termination_status(m))
x = value.(x)

for t in T
    for c in C
        if x[c, t] > 0
            println("Semester ", t, ": take \"", c, ": ", cls_data[c]["title"], "\"")
        end
    end
end

OPTIMAL
Semester 1: take "COMP SCI 200: Programming I"
Semester 1: take "COMP SCI 425: Introduction to Combinatorial Optimization"
Semester 1: take "COMP SCI 240: Introduction to Discrete Mathematics"
Semester 1: take "COMP SCI 252: Introduction to Computer Engineering"
Semester 2: take "COMP SCI 300: Programming II"
Semester 3: take "COMP SCI 400: Programming III"
Semester 3: take "COMP SCI 354: Machine Organization and Programming"
Semester 3: take "COMP SCI 407: Foundations of Mobile Systems and Applications"
Semester 4: take "COMP SCI 520: Introduction to Theory of Computing"
Semester 4: take "COMP SCI 407: Foundations of Mobile Systems and Applications"


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=0bcaab0a-12b1-4bbe-9d8b-27e52aa08424' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>